# Indeed Job Scraper

## Steps

1. Get all of the URL's of EACH LISTING. for each page teh MAIN function gets, get the LINKS for that page (the job postings). For example 15 pages gets a list of 150 URLS. 
- for EACH page
    - get Mosaic job card link
    - iterate through the list of job cards
    - append list of URLS by each individual URL
    - move to next page in Indeed
2. Look through the entire URL list and process them individually.
- for each URL
- extract the parts of the page for features 
3. On each individual page, 'write a row' for the data about the job as each features.

Loop through list of URLs
for each URLs, append a list by a dictionary of its contents
dump into pd df
pd.to_pickle('filepath')

In [6]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from time import sleep
from random import randint
from datetime import datetime
import pandas as pd

In [7]:
def get_url(position, location,page_count):
        if page_count > 0:
            template = 'https://www.indeed.com/jobs?q={}&l={}&start={}'
            page_count *= 10
            url = template.format(position, location,page_count)
        else:
            template = 'https://www.indeed.com/jobs?q={}&l={}'
            url = template.format(position, location)
        
        return 'http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=' + url

def job_listings_generator(position, location,page_count=10):
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
}
    
    urls = [] 
    i = 0
    
    # POPULATE LIST OF URLs
    
    while i < page_count:
        
        url = get_url(position, location,i)  # create the url while passing in the position and location.
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find( id = "mosaic-zone-jobcards" )
        
        if cards is None:
            print('EMPTY URL HERE! ')
            print(url)
            time.sleep(6.8)
            continue
            
        cards = cards.find_all('a')
        cards_test = [card for card in cards if 'pagead' in card['href'] or '/rc/' in card['href']]
        
        for card in cards:
            record = card['href']


            if "pagead" in record:
                urls.append("https://www.indeed.com"+record)

            if "/rc/" in record:
                urls.append("https://www.indeed.com"+record)
        
        time.sleep(0.1)
        i += 1
    
    return urls


In [8]:
def job_info(url,title,location):
    try:
        response = requests.get(url)
    except Exception as e:
        print(str(e))
        time.sleep(5.0)
        try:
            print('trying again!')
            response = requests.get(url)
        except Exception as e:
            return {}
        
    soup = BeautifulSoup(response.text, 'html.parser')
    d = dict()
    
    
    #location
    d["location"]=location
    
    #title
    d["job_title"]=title

    
    job_info = soup.find(class_="jobsearch-ViewJobLayout-jobDisplay icl-Grid-col icl-u-xs-span12 icl-u-lg-span7" )
    
#     print(job_info)
    
    
    if job_info is None:
        d["job_desc"] = ""
        time.sleep(6.8)
        return d
        
    cards = job_info.find_all('h2')
    
    if len(cards) == 0:
        d["job_desc"] = ""
        time.sleep(6.8)
        return d
            

    #description
    for card in cards:
        d["job_desc"]=card.find_next('div').text

    return d

In [ ]:
# urls = []
# locations = ["major metros / areas"]
# titles = ["10+ job titles"]

titles = ["data+scientist","financial+analyst", "underwriter","chemical+engineer","physician","university+recruiter"]
locations = ["San+Francisco","Portland", "Seattle","New+York","Los+Angeles","Austin"]

urls = []
job_information = []


for title in titles:
    for location in locations:
        urls=job_listings_generator(title,location,8)
        for job in urls:
            job_information.append(job_info(job,title,location))
            
            time.sleep(0.1)
            
        print(f"finished {location} listings for {title}!")
            
            
            
    



In [ ]:
df = pd.DataFrame(job_information)
df.head(10)

In [14]:
import os
df.to_pickle(os.getcwd()+'/webscraped_pickle')
df.to_csv('New_scrape')

NameError: name 'df' is not defined